In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -U transformers datasets peft accelerate bitsandbytes evaluate sacrebleu sacremoses --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 109.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 34.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/samanantar", "ta")
dataset["train"] = dataset["train"].select(range(50000))  # first 50k examples


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5264867 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer
model_name = "Helsinki-NLP/opus-mt-en-dra"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    inputs = [">>tam<< " + src for src in examples["src"]]
    targets = examples["tgt"]

    model_inputs = tokenizer(
        inputs, max_length=128, padding="max_length", truncation=True
    )
    labels = tokenizer(
        text_target=targets, max_length=128, padding="max_length", truncation=True
    )
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label_seq]
        for label_seq in labels["input_ids"]
    ]
    return model_inputs


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/818k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

In [4]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)


2025-08-09 09:27:04.130007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754731624.342826     138 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754731624.404006     138 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

In [8]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    report_to="tensorboard",   # "none" if you don’t want logs
    logging_dir="./logs",
    fp16=True
)


In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,3.442200,3.185384
2,3.331000,3.115635
3,3.319800,3.093492


TrainOutput(global_step=16875, training_loss=3.4654998191550925, metrics={'train_runtime': 1632.0799, 'train_samples_per_second': 82.717, 'train_steps_per_second': 10.34, 'total_flos': 4637432217600000.0, 'train_loss': 3.4654998191550925, 'epoch': 3.0})

In [10]:
%load_ext tensorboard
%tensorboard --logdir ./logs


<IPython.core.display.Javascript object>

In [11]:
import os
print(os.listdir("./logs"))


['events.out.tfevents.1754731747.749abd39bcc6.138.0']


In [12]:
model.save_pretrained("/kaggle/working/checkpoint")
tokenizer.save_pretrained("/kaggle/working/checkpoint")


('/kaggle/working/checkpoint/tokenizer_config.json',
 '/kaggle/working/checkpoint/special_tokens_map.json',
 '/kaggle/working/checkpoint/vocab.json',
 '/kaggle/working/checkpoint/source.spm',
 '/kaggle/working/checkpoint/target.spm',
 '/kaggle/working/checkpoint/added_tokens.json')

In [13]:
!zip -r checkpoint.zip /kaggle/working/checkpoint


  adding: kaggle/working/checkpoint/ (stored 0%)
  adding: kaggle/working/checkpoint/adapter_config.json (deflated 56%)
  adding: kaggle/working/checkpoint/README.md (deflated 66%)
  adding: kaggle/working/checkpoint/special_tokens_map.json (deflated 35%)
  adding: kaggle/working/checkpoint/source.spm (deflated 51%)
  adding: kaggle/working/checkpoint/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/checkpoint/target.spm (deflated 60%)
  adding: kaggle/working/checkpoint/vocab.json (deflated 76%)
  adding: kaggle/working/checkpoint/tokenizer_config.json (deflated 68%)


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_dir = "/kaggle/working/checkpoint"

# Load tokenizer and model from checkpoint folder
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir)

# Example English sentences to translate
sentences = [
    "Hello, how are you?",
    "This is a test sentence.",
    "I am fine tuning a translation model on Kaggle."
]

inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

# Generate translations
outputs = model.generate(**inputs, max_length=128)
translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print translated sentences
for src, tgt in zip(sentences, translations):
    print(f"Input: {src}")
    print(f"Translation: {tgt}")
    print()


Input: Hello, how are you?
Translation: ஹலோ, ఎలా ఉన్నావు?

Input: This is a test sentence.
Translation: இது ஒரு சோதனைத் தீர்ப்பு.

Input: I am fine tuning a translation model on Kaggle.
Translation: Kaggle on Windown Cagle ನ ಭಾಷಾಂತರ ನಮೂನೆಯನ್ನು ಸರಿಯಾಗಿ ಅರ್ಥನಿರೂಪಿಸುತ್ತಿದ್ದೇನೆ.



In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_dir = "/kaggle/working/checkpoint"

# Load tokenizer and model checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir)

# Your English sentences with appropriate target language prefix
sentences = [
    "Hello, how are you?",
    "This is a test sentence.",
    "I am fine tuning a translation model on Kaggle."
]

# Add the Tamil language token prefix as used during training
inputs = tokenizer([">>tam<< " + s for s in sentences], return_tensors="pt", padding=True, truncation=True)

# Generate translations with model
outputs = model.generate(**inputs, max_length=128)

# Decode the outputs skipping special tokens
translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print final translations (all should be in Tamil)
for src, tgt in zip(sentences, translations):
    print(f"Input: {src}")
    print(f"Translation: {tgt}")
    print()


Input: Hello, how are you?
Translation: ஹலோ, எப்படி இருக்கிறாய்?

Input: This is a test sentence.
Translation: இது ஒரு சோதனைத் தீர்ப்பு.

Input: I am fine tuning a translation model on Kaggle.
Translation: கக்லேயில் ஒரு மொழிபெயர்ப்பு மாதிரியை நடித்துக்கொண்டு வருகிறேன்.

